# Apply a bone tumor segmenter on an radiomic exam

In [1]:
# imports
import ipywidgets as widgets
from PIL import Image
import io
import numpy as np

from detectron2.utils.visualizer import ColorMode
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo
from detectron2.data import DatasetCatalog, MetadataCatalog

register_coco_instances(
    "my_dataset_train", {}, "./train.json", ''
)

from IPython.display import HTML
loading = HTML('<img src="img/loading.gif">')

In [4]:
cfg = get_cfg()

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.DATALOADER.SAMPLER_TRAIN = "RepeatFactorTrainingSampler"
cfg.DATALOADER.REPEAT_THRESHOLD = 0.001
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = './model.pth'
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.MODEL.DEVICE= 'cpu'
mal_col = (0, 69, 255)  # red
ben_col = (50, 205, 50)  # green


MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_colors = [ben_col, mal_col]
predictor = DefaultPredictor(cfg)
#cfg.DATASETS.TRAIN

### 1. Upload the desired radiograph in '.png' format

In [5]:
btn_upload = widgets.FileUpload()
btn_upload

FileUpload(value={}, description='Upload')

In [6]:
def on_data_change(change):
    out_pl.clear_output()
    with out_pl: display(loading)
    img = Image.open(io.BytesIO(btn_upload.data[-1]))
    out_pl.clear_output()
    with out_pl: display(img.resize([512, 512]))

btn_upload.observe(on_data_change, names=['data'])

In [7]:
out_pl = widgets.Output()
out_pl.clear_output()
out_pl

Output()

In [12]:
btn_run = widgets.Button(description='Segment')
btn_run

Button(description='Segment', style=ButtonStyle())

In [9]:
def on_click_segment(change):
    out_pl.clear_output()
    with out_pl: display(loading)
    img = np.array(Image.open(io.BytesIO(btn_upload.data[-1])))
    
    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")[:1]
    
    v = Visualizer(
        img[:, :, ::-1],
        metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]),
        scale=1,
        instance_mode=ColorMode.SEGMENTATION,
    )
    v = v.draw_instance_predictions(instances)
    back_img = Image.fromarray(v.get_image()[:, :, ::-1]) 
    
    out_pl.clear_output()
    with out_pl: display(back_img.resize([512, 512]))

btn_run.on_click(on_click_segment)

In [10]:
#!pip install voila
#!jupyter serverexternsion enable voila --sys-prefix